In [2]:
import sys
import polars as pl
from datetime import date, timedelta, datetime, timezone
import numpy as np

sys.path.append("../v3-polars/")
from v3 import state

In [5]:
poolAddress = '0xbf7d01d6cddecb72c2369d1b421967098b10def7'
update = True

pool = state.v3Pool(poolAddress, 'ethereum', update = update)

swaps = pool.swaps
mb = pool.mb

Starting table factory_pool_created
Found 12369739 to 19669984
Found data
Updated to 19669985 to 19669984
Nothing to update
Starting table pool_swap_events
Found 12369879 to 19669985
Found data
Updated to 19662557 to 19669985
Starting at 19662557
Going from 19662557 to 19669985
Starting table pool_mint_burn_events
Found 12369739 to 19664336
Found data
Updated to 19657199 to 19664336
Starting at 19657199
Going from 19657199 to 19664336
Starting table pool_initialize_events
Found 12369739 to 19658165
Found data
Updated to 19650824 to 19658165
Starting at 19650824
Going from 19650824 to 19658165


In [6]:
_ = '''
TODO

- I am checking liquidity by a manufactured key (which may get transferred)
I need to calculate the key if not nft position manager
This can be solved by using tokenid

- We distribute as of last swap in the block
However, this may pick up JIT liquidity. We can calculate as_of the end of the block,
but this will mean we cannot check that we are arriving at the correct number
However, I am confident that we are

- I am only distributing rewards for the creator of a position
We could add a check for transfers to the mb frame
'''

In [7]:
# key = (from_address-tick_lower-tick_upper)
# this is how core indexes positions but uses keccack(key) instead
mb = mb.with_columns(key = (pl.col('from_address') + "-" + 
                       pl.col('tick_lower').cast(pl.Utf8) + "-" + 
                       pl.col('tick_upper').cast(pl.Utf8)),
                     liquidity_delta = pl.col('type_of_event') * pl.col("amount")
                    )

In [48]:
t_0 = datetime(year = 2024, month = 3, day = 1)
t_1 = datetime(year = 2024, month = 4, day = 14)

interval = timedelta(days = 30)
for t_0 in pl.date_range(t_0, t_1, interval = interval, eager = True):
    
    t_1 = t_0 + interval
    
    tgt_swaps = (swaps
            .filter(pl.col('block_timestamp') <= t_1.replace(tzinfo = timezone.utc))
            .sort('block_number')
            .select('block_number', 'log_index', 'block_timestamp', 'transaction_index', 
                     'tick', 'liquidity')
            )

    tgt_mb = (
            mb
            .filter(pl.col('block_timestamp') <= t_1.replace(tzinfo = timezone.utc))
            .filter(pl.col('block_number') >= tgt_swaps['block_number'].min())
             .sort('block_number')
            .select('block_number', 'log_index', 'block_timestamp', 'transaction_index')
            .with_columns(tick = pl.lit(None))
            )


    tgts = (pl.concat([tgt_swaps, tgt_mb], how = 'diagonal_relaxed')
            .sort(by = ['block_number', 'transaction_index', 'log_index'])
           )
    
    tgts = (tgts.join(
                # find the last transaction in the block
                (tgts 
                    .select(['block_number', 'transaction_index'])
                    .group_by('block_number')
                    .last()
                    .rename({"transaction_index": "last_index"})
                ),
                on = 'block_number', how = 'inner'
            )
            .filter(pl.col("last_index") == pl.col('transaction_index'))
            )
    
    tgts = (tgts.join(
                # find the last transaction in the block
                (tgts 
                    .select(['block_number', 'log_index'])
                    .group_by('block_number')
                    .last()
                    .rename({"log_index": "last_log"})
                ),
                on = 'block_number', how = 'inner'
            )
            .filter(pl.col("last_log") == pl.col('log_index'))
            .with_columns(twal=pl.col("block_timestamp").diff().dt.seconds(),
                          tick = pl.col('tick').forward_fill())
            )
    
    subset = tgts.filter(pl.col('block_timestamp') >= t_0.replace(tzinfo = timezone.utc))
    
    if subset.is_empty():
        subset = tgts.tail(1)
        
    data = []

    iterator = (
        subset
        .select(['block_number', 'transaction_index', 
                 'tick', 'liquidity', 'twal'])
        .sort('block_number')
    )

    missing_liquidity = False
    for (bn, tx_index, tick, liquidity, twal) in iterator.iter_rows():
        missing_liquidity = (liquidity == None)

        tick = int(tick)

        # decimalized version of block, index
        as_of = bn + tx_index / 1e4

        # calculate all lps in range as of the period we want
        lps = (mb
                 .filter((pl.col("as_of") < as_of) &
                         # positions are in range if tl <= tick < tu
                         (pl.col("tick_lower") <= tick) & 
                         (pl.col('tick_upper') > tick)
                        )
                 .select(['key', 'liquidity_delta'])
                 .group_by('key')
                 .sum()
                 # filter out the empty positions
                 .filter(pl.col("liquidity_delta") != 0)
                )

        # TODO
        # this is possible if there was a transfer of liquidity 
        # we could instead check via tokenID (which cannot change)
        assert lps.filter(pl.col('liquidity_delta') < 0).is_empty(), "Negative liquidity"

        # we know that all lps are positive and in-range
        tracked_liquidity = 0
        lp_dict = {}

        # calculate their shares
        for key, liquidity_delta in lps.iter_rows():
            tracked_liquidity += liquidity_delta

            lp_dict[key] = liquidity_delta

        # we know that all liquidity is tracked and their shares    
        # most likely issue here is a floating point error
        if not missing_liquidity:
            liquidity = int(liquidity)
            assert np.isclose(float(liquidity), float(tracked_liquidity)), "Missing liquidity"

        data.append([bn, twal, liquidity, lp_dict.copy()])
        
    shares = {}
    total_time = 0
    
    for bn, twal, liquidity, lps in data:
        total_time += twal
    
        for lp in lps.keys():
            # schema = (address, tl, tu)
            address = lp.split("-")[0]

            current_share = shares.get(address, 0)
            shares[address] = current_share + twal * (lps[lp] / tracked_liquidity)

        assert np.isclose(total_time, sum(shares.values())), 'Missing share of time'

    for lp in shares.keys():
        shares[lp] = shares[lp] / total_time
    
    print(shares)

/var/folders/rg/3f5pxvz936b5l2hmxgg3jhl80000gn/T/ipykernel_76377/3729155039.py:5: DeprecationWarning: Creating Datetime ranges using `date_range(s)` is deprecated. Use `datetime_range(s)` instead.
  for t_0 in pl.date_range(t_0, t_1, interval = interval, eager = True):


AssertionError: Missing share of time

In [52]:
total_time

1476

In [50]:
shares.values()

dict_values([8240.636037719229, 14866.348966987453])

In [42]:
lps

{}

In [154]:
tick, liquidity = int(tick), int(liquidity)

# decimalized version of block, index
as_of = bn + tx_index / 1e4

# calculate all lps in range as of the period we want
lps = (mb
         .filter((pl.col("as_of") <= as_of) &
                 # positions are in range if tl <= tick < tu
                 (pl.col("tick_lower") <= tick) & 
                 (pl.col('tick_upper') > tick)
                )
         .select(['key', 'liquidity_delta'])
         .group_by('key')
         .sum()
         # filter out the empty positions
         .filter(pl.col("liquidity_delta") != 0)
        )


In [18]:
liquidity

In [163]:
(
    tgts
            .join(
                # find the last transaction in the block
                (tgts 
                    .select(['block_number', 'transaction_index'])
                    .group_by('block_number')
                    .last()
                    .rename({"transaction_index": "last_index"})
                ),
                on = 'block_number', how = 'inner'
            )
     .filter(pl.col("last_index") == pl.col('transaction_index'))
    )

block_number,block_timestamp,transaction_index,tick,liquidity,liquidity_delta,twal,last_index
i64,"datetime[μs, UTC]",i64,str,str,f64,i64,i64
19317887,2024-02-27 09:06:35 UTC,151,"""265""","""36786839939111…",NaN,null,151
19317956,2024-02-27 09:20:35 UTC,52,"""266""","""36786839939111…",NaN,840,52
19317964,2024-02-27 09:22:11 UTC,57,"""272""","""36786839939111…",NaN,96,57
19317990,2024-02-27 09:27:23 UTC,83,"""277""","""36786839939111…",NaN,312,83
19318086,2024-02-27 09:46:47 UTC,78,"""278""","""36786839939111…",NaN,1164,78
19318110,2024-02-27 09:51:35 UTC,66,"""280""","""36786839939111…",NaN,288,66
19318230,2024-02-27 10:15:47 UTC,180,"""281""","""36786839939111…",NaN,1452,180
19318277,2024-02-27 10:25:11 UTC,115,"""303""","""36786839939111…",NaN,564,115
19318301,2024-02-27 10:29:59 UTC,124,"""308""","""36786839939111…",NaN,288,124


In [158]:
ld

-1.3330303255957763e+24

In [155]:
tick

113

In [156]:
liquidity

658897875134854891027

In [157]:
lps

key,liquidity_delta
str,f64
"""0x1724b8162669…",6.5890e20


In [43]:
sum(shares.values())

1.0

In [35]:
total_time

86844